# Feeding the DeepL model 

based on: 

https://keras.io/preprocessing/image/

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

## Load Model

Imports and Data Preprocessing have to be executed, then the loaded model can be used for the Testing and Visualisation steps

In [6]:
# load modules 
import os
from keras.models import load_model

#specify directory
os.chdir('C:\\Users\\lsamsi\\Documents\\GitHub\\DLfM_BrandManagement')

In [10]:
model = keras.models.load_model('./model/demo_model.h5')

In [21]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_5 (Dense)              (None, 12)                204       
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 52        
Total params: 592
Trainable params: 592
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
X_test = 

In [ ]:
model.predict(X_test)